In [367]:
!pip install mlxtend

In [368]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import dask.dataframe as dd
import dask as dask
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


names_trns = ["SKU","STORE","REGISTER","TRANNUM","INTERID","SALEDATE","STYPE","QUANTITY","AMT1","AMT2","AMT3","SEQ","MIC","extra"]
df_trans = dd.read_csv("/Users/mewtripiller/Desktop/Everything/IEMS 308/Assignment2/Dillards POS/trnsact.csv",names=names_trns)

names_str = ["STORE","CITY","STATE","ZIP","extra"]
df_str = pd.read_csv("/Users/mewtripiller/Desktop/Everything/IEMS 308/Assignment2/Dillards POS/strinfo.csv",names=names_str)


In [369]:
df_str.head(10)

,STORE,CITY,STATE,ZIP,extra
0,2,ST. PETERSBURG,FL,33710,0
1,3,ST. LOUIS,MO,63126,0
2,4,LITTLE ROCK,AR,72201,0
3,7,FORT WORTH,TX,76137,0
4,9,TEMPE,AZ,85281,0
5,60,ASPEN,CO,81611,0
6,62,TULSA,OK,74110,0
7,65,SAN ANTONIO,TX,78218,0
8,67,DALLAS,TX,75207,0
9,100,SALISBURY,NC,28146,0


In [370]:
df_trans.head(10)

,SKU,STORE,REGISTER,TRANNUM,INTERID,SALEDATE,STYPE,QUANTITY,AMT1,AMT2,AMT3,SEQ,MIC,extra
0,3,202,290,1100,326708721,2005-01-18,P,1,0.0,30.0,30.0,3500000,818,0
1,3,202,540,2700,326708721,2005-01-29,R,1,0.0,30.0,30.0,15200000,818,0
2,3,303,500,2100,23702074,2004-08-18,P,1,0.0,12.0,12.0,4600000,48,0
3,3,709,360,500,0,2005-08-14,P,1,0.0,30.0,30.0,6500000,818,0
4,3,802,660,400,0,2005-08-09,P,1,440.0,30.0,30.0,4700000,599,0
5,3,1202,400,2100,0,2004-11-11,P,1,0.0,30.0,30.0,8600000,999,0
6,3,1204,220,3400,0,2004-12-22,P,1,0.0,30.0,30.0,17600000,555,0
7,3,1304,160,3500,0,2004-08-07,P,1,0.0,30.0,30.0,11400000,990,0
8,3,1703,30,200,0,2005-08-09,P,1,440.0,30.0,30.0,4800000,999,0
9,3,1703,30,2400,0,2005-08-24,R,1,440.0,30.0,30.0,10100000,999,0


In [371]:
#select stores in Illinois
ILstores = df_str.STORE[df_str.STATE=="IL"]

In [372]:
ILstores

42      603
64     1003
251    4903
Name: STORE, dtype: int64

In [373]:
#select only columns we want
df_trans2 = df_trans.loc[:,["SKU","STORE","REGISTER","TRANNUM","SALEDATE"]]

In [374]:
df_trans2.head(10)

,SKU,STORE,REGISTER,TRANNUM,SALEDATE
0,3,202,290,1100,2005-01-18
1,3,202,540,2700,2005-01-29
2,3,303,500,2100,2004-08-18
3,3,709,360,500,2005-08-14
4,3,802,660,400,2005-08-09
5,3,1202,400,2100,2004-11-11
6,3,1204,220,3400,2004-12-22
7,3,1304,160,3500,2004-08-07
8,3,1703,30,200,2005-08-09
9,3,1703,30,2400,2005-08-24


In [375]:
#select only transactions that happened in IL
df_trans2 = df_trans2[df_trans2.STORE.isin(ILstores)]

In [376]:
#len(df_trans2)

In [377]:
#get into pandas
df_pd = df_trans2.compute()

In [378]:
#del df_final

In [379]:
df_final = df_pd

In [380]:
#form concatenated basket ID
df_final["concatID"]=df_final["STORE"].astype(str) + "_" + df_final["REGISTER"].astype(str) +"_" +df_final["TRANNUM"].astype(str)+"_" + df_final["SALEDATE"].astype(str)

In [381]:
#add basket ID
df_final['BASKETID'] = df_final.groupby("concatID").ngroup()

In [382]:
#remove all duplicate rows
df_final = df_final.drop_duplicates()

In [383]:
df_final.head(10)

,SKU,STORE,REGISTER,TRANNUM,SALEDATE,concatID,BASKETID
327,164,603,330,3900,2005-08-13,603_330_3900_2005-08-13,325375
409,164,4903,480,1300,2005-08-05,4903_480_1300_2005-08-05,211727
686,268,603,60,1400,2005-02-21,603_60_1400_2005-02-21,396174
694,268,1003,13,94600,2004-11-04,1003_13_94600_2004-11-04,16515
1798,450,603,580,1000,2005-08-01,603_580_1000_2005-08-01,383801
1799,450,603,580,1800,2005-08-03,603_580_1800_2005-08-03,385577
1800,450,603,600,5300,2005-08-20,603_600_5300_2005-08-20,394846
1837,450,1003,190,2200,2005-06-16,1003_190_2200_2005-06-16,27888
1838,450,1003,660,2800,2005-07-13,1003_660_2800_2005-07-13,83945
1839,450,1003,700,400,2005-08-25,1003_700_400_2005-08-25,92882


In [384]:
#len(df_final)

In [385]:
#filter by baskets that have more than 1 product in them
BASKETcount = df_final.groupby("BASKETID").count().STORE
selectBasks = BASKETcount.index[BASKETcount>1]

In [386]:
df_final2 = df_final[df_final.BASKETID.isin(selectBasks)]

In [387]:
#find 1000 most common SKUs and store them in selectSKUs
SKUcount = df_final2.groupby("SKU").count().STORE
selectSKUs = SKUcount.sort_values(ascending=False).iloc[0:1000,].index

In [388]:
df_final3 = df_final2[df_final2.SKU.isin(selectSKUs)]

In [389]:
df_final3

,SKU,STORE,REGISTER,TRANNUM,SALEDATE,concatID,BASKETID
66138,7915,603,80,800,2005-03-11,603_80_800_2005-03-11,408577
66141,7915,603,190,700,2005-07-02,603_190_700_2005-07-02,275752
66142,7915,603,240,200,2005-01-22,603_240_200_2005-01-22,282050
66144,7915,603,240,700,2004-11-07,603_240_700_2004-11-07,288304
66145,7915,603,240,900,2004-12-04,603_240_900_2004-12-04,289541
...,...,...,...,...,...,...,...
394928,9999950,4903,420,2500,2004-10-20,4903_420_2500_2004-10-20,203675
394930,9999950,4903,430,300,2005-07-23,4903_430_300_2005-07-23,206153
394932,9999950,4903,430,1200,2005-06-24,4903_430_1200_2005-06-24,204682
394933,9999950,4903,430,1400,2005-07-13,4903_430_1400_2005-07-13,204915


In [390]:
#filter again for baskets size >1
BASKETcount2 = df_final3.groupby("BASKETID").count().STORE
selectBasks2 = BASKETcount2.index[BASKETcount2>1]

In [391]:
df_final4 = df_final3[df_final3.BASKETID.isin(selectBasks2)]

In [392]:
df_final4.sort_values(by="BASKETID")

,SKU,STORE,REGISTER,TRANNUM,SALEDATE,concatID,BASKETID
423016,2076689,1003,100,2000,2005-03-02,1003_100_2000_2005-03-02,1743
201849,9686202,1003,100,2000,2005-03-02,1003_100_2000_2005-03-02,1743
41126,5930076,1003,100,2800,2005-04-01,1003_100_2800_2005-04-01,2815
14248,6490080,1003,100,2800,2005-04-01,1003_100_2800_2005-04-01,2815
610894,1978685,1003,100,500,2005-04-16,1003_100_500_2005-04-16,4452
...,...,...,...,...,...,...,...
45579,4148364,603,80,9900,2005-02-26,603_80_9900_2005-02-26,408683
676621,6614891,603,90,400,2005-01-20,603_90_400_2005-01-20,411017
507523,6324891,603,90,400,2005-01-20,603_90_400_2005-01-20,411017
12817,4526182,603,90,5200,2005-01-15,603_90_5200_2005-01-15,411445


In [393]:
df_assoc = pd.get_dummies(df_final4['SKU'],prefix='SKU')

In [394]:
df_assoc.insert(0,"BASKET",df_final4.BASKETID,True)
#df_assoc2 = pd.concat([df_assoc,onehot],axis=1)

In [395]:
finalData = df_assoc.groupby(['BASKET']).sum()

In [396]:
finalData

,SKU_7915,SKU_9633,SKU_10896,SKU_19171,SKU_19633,SKU_28371,SKU_29171,SKU_29633,SKU_29951,SKU_39171,...,SKU_9919950,SKU_9930064,SKU_9950895,SKU_9957390,SKU_9960064,SKU_9978362,SKU_9979950,SKU_9992306,SKU_9999170,SKU_9999950
BASKET,,,,,,,,,,,,,,,,,,,,,
1743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2815,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4452,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5897,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5913,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
408681,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
408683,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [397]:
frequentItemsets = apriori(finalData, min_support=0.0005, use_colnames=True)

In [398]:
frequentItemsets

,support,itemsets
0,0.002145,(SKU_7915)
1,0.001465,(SKU_9633)
2,0.001779,(SKU_10896)
3,0.001831,(SKU_19171)
4,0.001884,(SKU_19633)
...,...,...
1312,0.000628,"(SKU_6656135, SKU_7596135, SKU_6696135)"
1313,0.000576,"(SKU_6656135, SKU_7596135, SKU_6706135)"
1314,0.000523,"(SKU_6656135, SKU_7636135, SKU_7596135)"
1315,0.000576,"(SKU_7468362, SKU_7848362, SKU_7568362)"


In [399]:
rules = association_rules(frequentItemsets, metric="lift",min_threshold=1)

In [400]:
print(rules)

                    antecedents                 consequents  \
0                 (SKU_3884099)                 (SKU_39171)   
1                   (SKU_39171)               (SKU_3884099)   
2                 (SKU_3537981)                 (SKU_42062)   
3                   (SKU_42062)               (SKU_3537981)   
4                  (SKU_106343)               (SKU_5453386)   
..                          ...                         ...   
869  (SKU_8132644, SKU_8142644)               (SKU_8122644)   
870  (SKU_8142644, SKU_8122644)               (SKU_8132644)   
871               (SKU_8132644)  (SKU_8142644, SKU_8122644)   
872               (SKU_8122644)  (SKU_8132644, SKU_8142644)   
873               (SKU_8142644)  (SKU_8132644, SKU_8122644)   

     antecedent support  consequent support   support  confidence        lift  \
0              0.004029            0.002459  0.000523    0.129870   52.810169   
1              0.002459            0.004029  0.000523    0.212766   52.810169   


In [401]:
potentials = rules.antecedents[::-1]


In [402]:
potentials


873                 (SKU_8142644)
872                 (SKU_8122644)
871                 (SKU_8132644)
870    (SKU_8142644, SKU_8122644)
869    (SKU_8132644, SKU_8142644)
                  ...            
4                    (SKU_106343)
3                     (SKU_42062)
2                   (SKU_3537981)
1                     (SKU_39171)
0                   (SKU_3884099)
Name: antecedents, Length: 874, dtype: object

In [422]:
#find SKUs with highest lift, store in vals
vals= []
for k in potentials.index:
    check = potentials[k]
    if len(check)==1:
        x, = potentials[k]
        x2 = x[4:]
        if x not in vals:
            vals.append(x2)
    

In [424]:
#store 100 SKUs
SKUs = vals[0:100]

In [428]:
SKUs

['8142644',
 '8122644',
 '8132644',
 '7568362',
 '7848362',
 '7468362',
 '7596135',
 '7636135',
 '6656135',
 '6706135',
 '7596135',
 '6656135',
 '6696135',
 '7596135',
 '6656135',
 '6642521',
 '6742521',
 '6752521',
 '6320353',
 '6340353',
 '6300353',
 '6208362',
 '6738362',
 '6998362',
 '6032521',
 '6072521',
 '6062521',
 '5846627',
 '5826627',
 '5786627',
 '4898362',
 '5168362',
 '4648362',
 '4898362',
 '4318362',
 '4648362',
 '3978011',
 '6318344',
 '4108011',
 '3898011',
 '3968011',
 '4440924',
 '3898011',
 '3968011',
 '3690654',
 '3978011',
 '3898011',
 '3524026',
 '3908011',
 '3998011',
 '2726578',
 '2716578',
 '3988011',
 '3908011',
 '3898011',
 '3968011',
 '2716578',
 '2001637',
 '5487088',
 '1821637',
 '1751939',
 '1671939',
 '2141939',
 '803921',
 '3898011',
 '3968011',
 '803921',
 '3690654',
 '3968011',
 '9576432',
 '9716432',
 '9469364',
 '9600684',
 '9459364',
 '9590684',
 '8966664',
 '8646664',
 '8618636',
 '8798636',
 '8618636',
 '8718362',
 '8308357',
 '8288357',
 '8156